In [1]:
import pandas as pd
import seaborn as sns

veri = sns.load_dataset('titanic')
veri.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [2]:
print(veri.isnull().sum())

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64


In [3]:
veri = veri.drop(['deck', 'embark_town', 'alive', 'who', 'adult_male', 'class'], axis=1)
veri['age'] = veri['age'].fillna(veri['age'].mean())
veri = veri.dropna()
print(veri.isnull().sum())

survived    0
pclass      0
sex         0
age         0
sibsp       0
parch       0
fare        0
embarked    0
alone       0
dtype: int64


In [5]:
veri = pd.get_dummies(veri, columns=['sex', 'embarked'], drop_first=True)
veri.head()

,survived,pclass,age,sibsp,parch,fare,alone,sex_male,embarked_Q,embarked_S
0,0,3,22.0,1,0,7.2500,False,1,0,1
1,1,1,38.0,1,0,71.2833,False,0,0,0
2,1,3,26.0,0,0,7.9250,True,0,0,1
3,1,1,35.0,1,0,53.1000,False,0,0,1
4,0,3,35.0,0,0,8.0500,True,1,0,1


In [6]:
y = veri['survived']
X = veri.drop('survived', axis=1)


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape)


(711, 9) (178, 9)


In [9]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
print("Model eğitildi.")

Model eğitildi.


In [10]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
y_pred = model.predict(X_test)
dogruluk = accuracy_score(y_test, y_pred)
print("\nModel Doğruluk Oranı:\n", dogruluk)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\n")
print("Classification Report:\n", classification_report(y_test, y_pred))



Model Doğruluk Oranı:
 0.797752808988764
Confusion Matrix:
 [[89 20]
 [16 53]]


Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.82      0.83       109
           1       0.73      0.77      0.75        69

    accuracy                           0.80       178
   macro avg       0.79      0.79      0.79       178
weighted avg       0.80      0.80      0.80       178



In [14]:
import pandas as pd

# --- BURAYI KENDİNE GÖRE DÜZENLE ---
benim_sinif = 3      # 1: Zengin, 2: Orta, 3: Gariban
yasim = 21           # Yaşın
kardes_es = 1        # Yanındaki kardeş/eş sayısı
ebeveyn_cocuk = 0    # Yanındaki anne-baba/çocuk sayısı
bilet_ucreti = 100   # Cebindeki para
cinsiyet = 0         # 1: Erkek, 0: Kadın
# -----------------------------------

# Yalnız olup olmadığını otomatik hesaplayalım (Model bunu istiyor)
yalniz_mi = (kardes_es == 0) and (ebeveyn_cocuk == 0)

# Veriyi hazırlayalım
benim_verim = pd.DataFrame({
    'pclass': [benim_sinif],
    'age': [yasim],
    'sibsp': [kardes_es],
    'parch': [ebeveyn_cocuk],
    'fare': [bilet_ucreti],
    'alone': [yalniz_mi],  # <-- İŞTE EKSİK OLAN BU SÜTUNDU
    'sex_male': [cinsiyet],
    'embarked_Q': [0],
    'embarked_S': [1]
})

# Sütun sırasının eğitimdekiyle aynı olduğundan emin olmak için ufak bir trik:
# (Eğer hata devam ederse bu kısım hayat kurtarır)
benim_verim = benim_verim[X_train.columns]

# Şimdi Tahmin Yap
sonuc = model.predict(benim_verim)
olasilik = model.predict_proba(benim_verim)

if sonuc[0] == 1:
    print(f"🎉 TEBRİKLER! Hayatta kaldın. (Yaşama ihtimalin: %{olasilik[0][1]*100:.1f})")
else:
    print(f"💀 MAALESEF... Buzlu sulara gömüldün. (Yaşama ihtimalin: %{olasilik[0][1]*100:.1f})")

🎉 TEBRİKLER! Hayatta kaldın. (Yaşama ihtimalin: %71.6)
